In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as psf

import os
#test change again

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1605539343869_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("DataOps").config("hive.metastore.client.factory.class","com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory").enableHiveSupport().getOrCreate()
sqlContext = SQLContext(spark)
spark.sparkContext.setLogLevel("ERROR")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
bronze_path = 's3://citi-bike-system-data-new/bronze/'
silver_path = 's3://citi-bike-system-data-new/silver/'
gold_path   = 's3://citi-bike-system-data-new/gold/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
trip_schema = StructType([
  StructField('tripduration', IntegerType()),
  StructField('start_time', StringType()),
  StructField('stop_time',  StringType()),
  StructField('start_station_id', IntegerType()),
  StructField('start_station_name', StringType()),
  StructField('start_station_latitude', StringType()),
  StructField('start_station_longitude', StringType()),
  StructField('end_station_id', IntegerType()),
  StructField('end_station_name', StringType()),
  StructField('end_station_latitude', StringType()),
  StructField('end_station_longitude', StringType()),
  StructField('bike_id', IntegerType()),
  StructField('user_type', StringType()),
  StructField('birth_year', StringType()),
  StructField('user_gender', StringType()),
  ])
  
# read the raw trip history data to dataframe, without triggering job, by passing csv schema
bronze_all_csv = bronze_path + '*'

bronzeDF = spark.read.csv(
  bronze_all_csv, 
  header=True,
  schema=trip_schema
  )
  
#bronze.rdd.getNumPartitions()
#bronze = bronze.coalesce(200) # If we use Coalesce to reduce NumPartitions, then we can use Coalesce to INCREASE numPart till that old num !!
#bronze = bronze.repartition(201, ["start_station_id", "end_station_id"])

bronzeDF = bronzeDF.distinct()
print(bronzeDF.orderBy(bronzeDF.start_time.desc()))

bronzeDF.write.format('parquet').mode('overwrite').save(silver_path)

slvrDF = spark.read.format('parquet').load(silver_path)

# Validation step
# Verify Counts/Schema/Top few records etc.
print("slvrDF.count()  :   ", slvrDF.count())
print("================================================")
print("slvrDF.dtypes  :   ", slvrDF.dtypes)
print("================================================")
print("slvrDF.head()  :   ", slvrDF.head())
print("================================================")
print("slvrDF.schema  :   ", slvrDF.schema)

slvrDF.createOrReplaceTempView("silver")

# Making DIM_STATION starts:
# 2 start cols:
startDF = slvrDF.select(["start_station_id","start_station_name"]).distinct().withColumnRenamed("start_station_id","station_id").withColumnRenamed("start_station_name","station_name")
# 2 end cols:
endDF = slvrDF.select(["end_station_id","end_station_name"]).distinct().withColumnRenamed("end_station_id","station_id").withColumnRenamed("end_station_name","station_name")
# Merge them into dim station DF:
unionDF = startDF.unionAll(endDF).distinct()

#display(unionDF)
unionDF.write.format('parquet').mode('overwrite').save(f"{gold_path}dim_station")

# Create Managed Table using spark SQL (alternative to Hive SQL)
tableName = "dim_station"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o114.csv.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: 5E87688D0BF834C2; S3 Extended Request ID: JxAT5cKMi4qwSnlx/cHQp1JWg9XTM3NNc4LM80UVfJR9kVPCerUdY9se9E8/yDCAqfEOnVRSdXQ=; Proxy: null), S3 Extended Request ID: JxAT5cKMi4qwSnlx/cHQp1JWg9XTM3NNc4LM80UVfJR9kVPCerUdY9se9E8/yDCAqfEOnVRSdXQ=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:303)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.listStatus(S3NativeFileSystem.java:606)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.listStatus(S3NativeFileSystem.java:577)
	at org.apache.hadoop.fs.Globber.listStatus(Globber.java:77)
	at org.apache.hadoop.fs.Globber.doGlob(Globber.java:235)
	at org.apache.hadoop.fs.Globber.glob(Globber.java:149)
	at org.apache.hadoop.fs.FileSystem.globStatus(File